In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.insert(0,'****/Functions_Classes')
import confounder_correction_classes_combat_modmean_lr_normal
from confounder_correction_classes_combat_modmean_lr_normal import ComBatHarmonization,StandardScalerDict, BiocovariatesRegression

#Datasets and system
import numpy as np                # Management of arrays
import os                         # System utils
import pandas as pd
from pathlib import Path          # path and file utils
from scipy.io import loadmat      # Required to load .mat files
import h5py #For creating new dataset files
import sklearn
import copy
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load Raw Data

In [ ]:
# Atlas regions names
data_path='****/Organized_Inputs/'

atlas_ROI_names=pd.read_csv(os.path.join(data_path, "atlas_ROI_sorted.csv"))

# HCP
HCP_covars_raw=pd.read_csv(os.path.join(data_path, "HCP_covar.csv"))
HCP_data_raw=pd.read_csv(os.path.join(data_path, "HCP_data.csv"))
n_HCP=HCP_data_raw.shape[0]

# test set
Strat_covars_raw=pd.read_csv(os.path.join(data_path, "test_set_YA_covar_raw.csv"))
Strat_data_raw=pd.read_csv(os.path.join(data_path, "test_set_YA_data_raw.csv"))

n_Strat=Strat_data_raw.shape[0]

# Globals

strat_globals_data=copy.deepcopy(Strat_covars_raw[['TIV','CT_avg','CT_std','abGM','abWM','abCSF']])
strat_covars=copy.deepcopy(Strat_covars_raw[['Batch','Age','Gender','Diagnosis','index']])

In [ ]:

HCP_covars_raw[['Batch']]=0
HCP_covars_raw[['Diagnosis']]=0

column_to_move = HCP_covars_raw.pop("Batch")
# insert column with insert(location, column_name, column_value)
HCP_covars_raw.insert(0, "Batch", column_to_move)
column_to_move = HCP_covars_raw.pop("Diagnosis")
# insert column with insert(location, column_name, column_value)
HCP_covars_raw.insert(3, "Diagnosis", column_to_move)
HCP_covars_raw

# Globals
hcp_globals_data=HCP_covars_raw[['TIV','CT_avg','CT_std','abGM','abWM','abCSF']].copy()
HCP_covars=HCP_covars_raw[['Batch','Age','Gender', 'Diagnosis']].copy()

In [ ]:
HCP_data_raw=HCP_data_raw.iloc[:,0:170].copy()
Strat_data_raw=Strat_data_raw.iloc[:,0:170].copy()

# CR pipeline TIV: M-ComBat

In [ ]:
# M-ComBat Variation: Data is harmonized to a reference batch, in this case the HCP HC dataset
ref_batch=0 # Is the HCP dataset

globals_id=np.arange(0,6)

# The TIV is harmonized together with globals
globals_of_interest={'globals': {'id': globals_id, 'categorical': ['Gender'],
                               'continuous':['Age']} }

# Multi-center TIV Harmonization
combat_ref_TIV=ComBatHarmonization(cv_method=None, ref_batch=ref_batch,
                                   regression_fit=0,feat_detail=globals_of_interest,
                                   feat_of_no_interest=None)

# Concatenate Strat HC and HCP bootstrap sample

# Concatenate all HC data together
HC_data=pd.concat([HCP_data_raw, Strat_data_hc],axis=0).reset_index(drop=True)
HC_covars=pd.concat([HCP_covars,strat_covars_hc],axis=0).reset_index(drop=True)
HC_covars['Batch']=HC_covars['Batch'].astype('int')
HC_covars.rename(columns={"Batch":"batch"}, inplace=True)
HC_globals=pd.concat([hcp_globals_data,strat_globals_hc],axis=0).reset_index(drop=True)

# Harmonized TIV

# Estimate combat center parameters for HC test set
HC_globals_dict={'data': HC_globals, 'covariates': HC_covars}

HC_globals_harm=combat_ref_TIV.fit_transform(HC_globals_dict)

# Apply combat center parameters to not_HC test set
strat_covars_nothc.rename(columns={"Batch":"batch"}, inplace=True)

notHC_globals_dict={'data': strat_globals_nothc, 'covariates': strat_covars_nothc}

notHC_globals_harm=combat_ref_TIV.transform(notHC_globals_dict)

# Extrat harmonized TIV
hc_harm_TIV=pd.DataFrame(HC_globals_harm[:,0], columns=['TIV'])
nothc_harm_TIV=pd.DataFrame(notHC_globals_harm[:,0], columns=['TIV'])

# Replace raw TIV covariate with harmonized TIV
HC_covars['TIV']=hc_harm_TIV
strat_covars_nothc['TIV']=nothc_harm_TIV


[0 1 2 3 4 5 6 7]
[neuroCombat] Creating design matrix
[neuroCombat] Standardizing data across features
[neuroCombat] Fitting L/S model and finding priors
[neuroCombat] Finding parametric adjustments
[neuroCombat] Final adjustment of data
Batchs for this set ['0' '1' '2' '3' '4' '5' '6' '7']
Batchs from the training set estimation ['0' '1' '2' '3' '4' '5' '6' '7']
M-ComBat option was used during fitting
Batches in current set matching the ones from training set estimation [0 1 2 3 4 5 6 7]
Transforming tthe reference and other sets used in the combat estimation
 Reintroducing original data for all data from reference site
Batchs for this set ['1' '2' '3' '4' '5' '6' '7']
Batchs from the training set estimation ['0' '1' '2' '3' '4' '5' '6' '7']
M-ComBat option was used during fitting
Batches in current set matching the ones from training set estimation [1 2 3 4 5 6 7]
Transforming a test set with reference estimations


In [ ]:
HCP_globals_harm=pd.DataFrame(HC_globals_harm[:1109], columns=['TIV','CT_avg','CT_std','abGM','abWM','abCSF'])
Strat_globals_harm=pd.concat([pd.DataFrame(HC_globals_harm[1109:]), pd.DataFrame(notHC_globals_harm)], axis=0)
Strat_globals_harm.columns=['TIV','CT_avg','CT_std','abGM','abWM','abCSF']

In [ ]:
main_path='****/Corrected_withglobals_modmean/combat_modmean_lr_betas/'

path= main_path + 'test_set_YA_globals_harm.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  pd.DataFrame(Strat_globals_harm).to_csv(f,index=False)

path= main_path + 'HCP_globals_harm.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  pd.DataFrame(HCP_globals_harm).to_csv(f,index=False)

# CR pipeline features : M-ComBat + LR

In [ ]:
# Brain features index-order definition
dk40_feat= np.arange(0,68) # 68 cortical thickness
cobravgm_feat=np.arange(68,118) # 52 gray matter volumes
cobravwm_feat=np.arange(118,170) # 52 white mater volumes
vol_feat=np.concatenate((cobravgm_feat,cobravwm_feat))

# Create the dictionaries with data + covariates for harmonization

HC_dict={'data': HC_data , 'covariates': HC_covars}

BD_dict={'data': Strat_data_nothc , 'covariates': strat_covars_nothc}

# Brain feature-types to consider separatly and associated bio-covariates
feat_of_interest={'cortical': {'id': dk40_feat, 'categorical': ['Gender'],
                               'continuous':['Age']},
                  'volumes': {'id': vol_feat, 'categorical': ['Gender'],
                              'continuous':['Age','TIV']}}

# Preprocessing pipelne: Regressing-out confounding biological covariates
ref_batch=0 # Is the HCP dataset

combat_ref_data=ComBatHarmonization(cv_method=None, ref_batch=ref_batch,
                                    regression_fit=1,feat_detail=feat_of_interest,
                                    feat_of_no_interest=None)

scaler_data_fun_1=StandardScalerDict(ref_batch=ref_batch,std_data=True, std_cov=False,
                                     output_dict=True)

scaler_data_fun_2=StandardScalerDict(ref_batch=ref_batch,std_data=True, std_cov=False,
                                     output_dict=False)

biocov_regre=BiocovariatesRegression(ref_batch=ref_batch,cv_method=None,
                                     feat_detail=feat_of_interest,
                                     output_dict=True)

combat_biocovregr_steps = [('Multi-center features Harmonization', combat_ref_data),
 ('Standardize only data 1', scaler_data_fun_1),('Confounder Regression', biocov_regre),
  ('Standardize only data 2', scaler_data_fun_2)]

preprocessing_pipeline = Pipeline(combat_biocovregr_steps)

# Fit_transform method in training set
HC_corr=preprocessing_pipeline.fit_transform(HC_dict)

# Transforming Test set
BD_corr=preprocessing_pipeline.transform(BD_dict)

# Put together test set Data
Strat_data_corr=pd.concat([pd.DataFrame(HC_corr[1109:]), pd.DataFrame(BD_corr)], axis=0)
Strat_covars=pd.concat([HC_covars.iloc[1109:], strat_covars_nothc], ignore_index=True)

HCP_data_corr=pd.DataFrame(HC_corr[:1109])
HCP_covars_corr=HC_covars.iloc[:1109].drop(labels=['index'], axis=1).copy()

[0 1 2 3 4 5 6 7]
[neuroCombat] Creating design matrix
[neuroCombat] Standardizing data across features
[neuroCombat] Fitting L/S model and finding priors
[neuroCombat] Finding parametric adjustments
[neuroCombat] Final adjustment of data
[0 1 2 3 4 5 6 7]
[neuroCombat] Creating design matrix
[neuroCombat] Standardizing data across features
[neuroCombat] Fitting L/S model and finding priors
[neuroCombat] Finding parametric adjustments
[neuroCombat] Final adjustment of data
Batchs for this set ['0' '1' '2' '3' '4' '5' '6' '7']
Batchs from the training set estimation ['0' '1' '2' '3' '4' '5' '6' '7']
M-ComBat option was used during fitting
Batches in current set matching the ones from training set estimation [0 1 2 3 4 5 6 7]
Transforming tthe reference and other sets used in the combat estimation
 Reintroducing original data for all data from reference site
Batchs for this set ['0' '1' '2' '3' '4' '5' '6' '7']
Batchs from the training set estimation ['0' '1' '2' '3' '4' '5' '6' '7']
M-C

# Save Data in .csv

In [ ]:
# Save Data - only harmonized with ref_batch=HCP
main_path='****/Corrected_withglobals_modmean/combat_modmean_lr_betas/'

path= main_path + 'test_set_YA_data_corr_withglobals.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  pd.DataFrame(Strat_data_corr).to_csv(f,index=False)

path= main_path + 'test_set_YA_covar_corr_withglobals.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  pd.DataFrame(Strat_covars).to_csv(f,index=False)

path= main_path + 'HCP_data_corr.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  pd.DataFrame(HCP_data_corr).to_csv(f,index=False)

# The covariates remain unchanged but we save it in the same folder for convinience
path= main_path + 'HCP_covars_corr.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  pd.DataFrame(HCP_covars_corr).to_csv(f,index=False)


# CR pipeline: only M-ComBat

In [ ]:

# Brain features index-order definition
dk40_feat= np.arange(0,68) # 68 cortical thickness
cobravgm_feat=np.arange(68,118) # 52 gray matter volumes
cobravwm_feat=np.arange(118,170) # 52 white mater volumes
vol_feat=np.concatenate((cobravgm_feat,cobravwm_feat))

# Create the dictionaries with data + covariates for harmonization

X_normative_dict={'data': HC_data , 'covariates': HC_covars}

BD_test_dict={'data': Strat_data_nothc , 'covariates': strat_covars_nothc}

# Brain feature-types to consider separatly and associated bio-covariates
feat_of_interest={'cortical': {'id': dk40_feat, 'categorical': ['Gender'],
                               'continuous':['Age']},
                  'volumes': {'id': vol_feat, 'categorical': ['Gender'],
                              'continuous':['Age','TIV']}}

# Preprocessing pipelne: Regressing-out confounding biological covariates
ref_batch=0 # Is the HCP dataset

combat_ref_data=ComBatHarmonization(cv_method=None, ref_batch=ref_batch,
                                    regression_fit=0,feat_detail=feat_of_interest,
                                    feat_of_no_interest=None)

# Fit_transform method in training set
X_normative_harm=combat_ref_data.fit_transform(X_normative_dict)

# Transforming Test set
BD_harm=combat_ref_data.transform(BD_test_dict)

# Put together test set Data
Strat_data_harm=pd.concat([pd.DataFrame(X_normative_harm[1109:]), pd.DataFrame(BD_harm)], axis=0)
Strat_covars_harm=pd.concat([HC_covars.iloc[1109:], strat_covars_nothc], ignore_index=True)

# Put together HCP data
HCP_data_harm=pd.DataFrame(X_normative_harm[:1109])
HCP_covars_harm=HC_covars.iloc[:1109].drop(labels=['index'], axis=1).copy()

[0 1 2 3 4 5 6 7]
[neuroCombat] Creating design matrix
[neuroCombat] Standardizing data across features
[neuroCombat] Fitting L/S model and finding priors
[neuroCombat] Finding parametric adjustments
[neuroCombat] Final adjustment of data
[0 1 2 3 4 5 6 7]
[neuroCombat] Creating design matrix
[neuroCombat] Standardizing data across features
[neuroCombat] Fitting L/S model and finding priors
[neuroCombat] Finding parametric adjustments
[neuroCombat] Final adjustment of data
Batchs for this set ['0' '1' '2' '3' '4' '5' '6' '7']
Batchs from the training set estimation ['0' '1' '2' '3' '4' '5' '6' '7']
M-ComBat option was used during fitting
Batches in current set matching the ones from training set estimation [0 1 2 3 4 5 6 7]
Transforming tthe reference and other sets used in the combat estimation
 Reintroducing original data for all data from reference site
Batchs for this set ['0' '1' '2' '3' '4' '5' '6' '7']
Batchs from the training set estimation ['0' '1' '2' '3' '4' '5' '6' '7']
M-C

# Save onlyharmonize data in .csv

In [ ]:
# Save Data - only harmonized with ref_batch=HCP
main_path='****/Corrected_withglobals_modmean/combat_modmean_lr_betas/'

path= main_path + 'test_set_YA_data_onlyharm_withglobals.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  pd.DataFrame(Strat_data_harm).to_csv(f,index=False)

path= main_path + 'test_set_YA_covar_onlyharm_withglobals.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  pd.DataFrame(Strat_covars_harm).to_csv(f,index=False)

path= main_path + 'HCP_data_onlyharm.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  pd.DataFrame(HCP_data_harm).to_csv(f,index=False)

# The covariates remain unchanged but we save it in the same folder for convinience
path= main_path + 'HCP_covars_onlyharm.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  pd.DataFrame(HCP_covars_harm).to_csv(f,index=False)